## One-Set Intro Mahjong Probability Calculations

This notebook tabulates _shanten_ probabilities for hands in an introductory version of Mahjong featuring the following properties:
- **Tiles**: 36 tiles from a single numeric suit (dots); 4 copies each of the numbers 1-9.
- **Hand**: A completed hand has five tiles, consisting of one set of three (sequence or triplet) and one pair.

In [2]:
import numpy as np
import pandas as pd

In [10]:
# load pre-computed tile combination properties
suited_df = pd.read_csv('./shanten_suuhai.csv', 
                        index_col='tile_int', 
                        dtype={'tile_vector': str})

# trim to only combinations with four or five tiles
active_df   = suited_df[suited_df['n_tiles'] == 5]
inactive_df = suited_df[suited_df['n_tiles'] == 4]

In [11]:
print(active_df.shape)
active_df.sample(5)

(1278, 11)


,tile_vector,n_tiles,n_sets,n_triplets,n_sequences,n_blocks,n_pairs,max_pairs,n_koritsu,n_terminals,n_ways
tile_int,,,,,,,,,,,
35777,001010300,5,1,1,0,1,0,0,0,0,64
27789,010000211,5,1,0,1,0,0,1,2,1,384
55789,000020111,5,1,0,1,1,1,1,0,1,384
22588,020010020,5,0,0,0,2,2,2,1,0,144
12388,111000020,5,1,0,1,1,1,1,0,1,384


In [12]:
print(inactive_df.shape)
inactive_df.sample(5)

(495, 11)


,tile_vector,n_tiles,n_sets,n_triplets,n_sequences,n_blocks,n_pairs,max_pairs,n_koritsu,n_terminals,n_ways
tile_int,,,,,,,,,,,
2445,010210000,4,0,0,0,2,0,1,0,0,96
6688,000002020,4,0,0,0,2,2,2,0,0,36
4458,000210010,4,0,0,0,1,1,1,1,0,96
1199,200000002,4,0,0,0,2,2,2,0,2,36
1466,100102000,4,0,0,0,1,1,1,1,1,96


In [4]:
def vector_to_int(t_vector):
    t_int = ''
    for i, cnt in zip(np.arange(1,len(t_vector)+1),t_vector):
        t_int += cnt * str(i)
    if t_int:
        return int(t_int)
    else:
        return 0

def int_to_vector(t_int, n_types=9):
    t_vector = np.zeros(n_types, dtype=int)
    t_int = str(t_int)
    for i in t_int:
        t_vector[int(i)-1] += 1
    return t_vector

### Hand Completion Probabilities

For hands with five tiles:
- How many possible hands are there?
- How many of those hands form a winning combination?

In [14]:
### How many possible hands are there, winning or otherwise?
total_hands = active_df['n_ways'].sum()
print(total_hands)

376992


In [17]:
### How many possible winning hands are there?
active_complete = active_df.query('(3 * n_sets + 2 * n_pairs == n_tiles) & (n_pairs == 1)')
winning_hands = active_complete['n_ways'].sum()

print(winning_hands)
print(f"proportion: {winning_hands/total_hands:0.7f}; 1 in {total_hands/winning_hands:.0f}")

19200
proportion: 0.0509295; 1 in 20


### Shanten Probabilities

For hands with four tiles:
- How many hands are there?
- What is the probability that a random hand is ready for completion (i.e. _tenpai_)?
- What is the probability that a random hand is one away from ready (i.e. _iishanten_)?

Due to the limited number of tiles, this is the maximum shanten count!

In [22]:
### How many possible hands are there?
total_hands = inactive_df['n_ways'].sum()
print(total_hands)

58905


In [33]:
### How many tenpai hands are there?
inactive_tenpai = inactive_df.query('(n_sets == 1) | (n_pairs == 2) | (n_blocks == 2 & n_pairs == 1)')
tenpai_hands = inactive_tenpai['n_ways'].sum()

print(tenpai_hands)
print(f"proportion: {tenpai_hands/total_hands:0.7f}; 1 in {total_hands/tenpai_hands:.2f}")

21753
proportion: 0.3692895; 1 in 2.71


In [46]:
### How many iishanten hands are there?
inactive_iishanten = inactive_df.query('(n_blocks == 2 & n_pairs == 0) | (n_blocks == 1)')
iishanten_hands = inactive_iishanten['n_ways'].sum()

print(iishanten_hands)
print(f"proportion: {iishanten_hands/total_hands:0.7f}; 1 in {total_hands/iishanten_hands:.2f}")

37152
proportion: 0.6307105; 1 in 1.59


In [47]:
avg_shanten = (0 * tenpai_hands + 1 * iishanten_hands)/total_hands
print(f"average shanten: {avg_shanten:.2f}")

average shanten: 0.63
